In [1]:
import pytraj as pt
import numpy as np
import pandas as pd
import tqdm
import sys
import gc
import copy
import nglview as nv

In [6]:
strucDir='structures/'
viewPDBname='Piezo_PIP2.prot_only.pdb'
trajName='Piezo_PIP2.prot_only.dcd'
topName='Piezo_PIP2.prot_only.parm7'
viewStruc=pt.load(strucDir+'Piezo_PIP2.prot_only.pdb')
iterTraj=pt.iterload(strucDir+trajName,
                     top=strucDir+topName)

print viewStruc
print iterTraj

pytraj.Trajectory, 1 frames: 
Size: 0.000000 (GB)
<Topology: 70296 atoms, 4254 residues, 12 mols, PBC with box type = ortho>
           
pytraj.TrajectoryIterator, 1592 frames: 
Size: 2.000000 (GB)
<Topology: 70296 atoms, 4254 residues, 12 mols, PBC with box type = ortho>
           


In [24]:
color_scheme=[
    ['blue','ARG LYS HIS HIP HID HIE HSD'],
    ['red','ASP GLU'],
    ['green','SER THR ASN GLN'],
    ['white','CYS SEC GLY PRO ALA VAL ILE LEU MET PHE TYR TRP']
]

In [68]:
view=nv.show_pytraj(viewStruc)
view.clear_representations()
view.add_representation('cartoon',color='white',alpha=.5)
view.add_representation('licorice',
                        color='blue',
                        selection='HSD LYS ARG')
view

NGLWidget()

In [80]:
from pytraj.utils.progress import ProgressBarTrajectory

In [86]:
basicResidueIDs=[res.original_resid for res in \
                 viewStruc.top.residues if res.name in ['HSD','LYS','ARG']]

commandList=['surf :%g'%iterTraj.top.residue(resID-1).original_resid \
             for resID in basicResidueIDs]
pbar=ProgressBarTrajectory(iterTraj,style='tqdm')
SASA_data=pt.compute(commandList,pbar)

KeyError: 0

In [95]:
basicResidueSASAtable=pd.DataFrame(np.arange(iterTraj.n_frames),columns=['Frame'])
basicResidueSASAtable['System']='Piezo_PIP2'
for ii,resID in tqdm.tqdm_notebook(enumerate(basicResidueIDs)):
    entryName='%s_%g'%(iterTraj.top.residue(resID-1).name,resID)
    basicResidueSASAtable[entryName]=SASA_data.values()[ii]
basicResidueSASAtable=basicResidueSASAtable[np.concatenate([
    ['System','Frame'],
    basicResidueSASAtable.columns
])]
basicResidueSASAtable.to_csv('basicResidueSASA.csv',index=False)
basicResidueSASAtable.head()

,Frame,System,ARG_1,ARG_15,ARG_21,ARG_22,HSD_27,LYS_30,LYS_42,ARG_63,...,HSD_4198,ARG_4210,LYS_4213,ARG_4222,ARG_4225,LYS_4236,ARG_4242,LYS_4249,ARG_4252,ARG_4254
0,0,Piezo_PIP2,231.301604,135.752731,90.688006,37.036016,81.810311,31.582857,158.942089,227.416121,...,41.210847,155.335757,111.492882,-5.209885,140.601920,84.719489,16.944188,148.135060,128.440404,290.796227
1,1,Piezo_PIP2,216.110266,174.780170,99.509969,16.703098,103.300599,72.856589,151.898330,210.569550,...,103.847289,162.340189,127.582129,37.842588,111.605389,113.826849,61.839898,158.491856,118.281692,288.793531
2,2,Piezo_PIP2,231.161291,155.527715,86.489593,9.983953,101.238810,65.590852,151.137150,200.432515,...,71.025217,139.332222,134.159950,26.783289,143.162468,152.197544,34.751632,179.815608,170.713722,316.306727
3,3,Piezo_PIP2,228.219199,141.413725,90.376183,4.560580,93.988088,28.210977,159.800871,207.038475,...,69.315142,107.660335,132.178808,19.361332,139.279869,145.529297,23.165717,200.181136,146.844401,289.914098
4,4,Piezo_PIP2,251.849992,146.331118,116.490590,12.492968,92.567033,54.396674,151.232110,217.081149,...,63.079660,107.718457,132.926795,41.318480,127.732068,165.243676,54.304498,192.992331,114.126485,273.409365
